In [1]:
import os
import random
import math

import numpy as np
import pandas as pd

### camera name

In [2]:
bounds = [(95, 25), (35, -35), (-25, -95), (155, 85), (-145, 145), (-85, -155)]
bounds_coef = [(math.tan(math.pi*(y1/180)), math.tan(math.pi*(y2/180))) for (y1, y2) in bounds]
color_list = ['b', 'g', 'orange', 'c', 'm', 'y', 'k', 'w', 'r']
camera_name = ['front_left', 'front', 'front_right', 'back_left', 'back', 'back_right']
bounds_quat = [[3,1], [1,4], [4,3], [2,1], [3,2], [4,3]]

In [3]:
def check_cam_index(index, camera_index):
    points = [(anno.iloc[index, 4+i], anno.iloc[index, 8+i]) for i in range(4)]
#     print(np.sign(points))
    for x, y in points:
        
        if np.sign(x) == 1 and np.sign(y) == 1:
            quat = 1
        elif np.sign(x) == 1 and np.sign(y) == -1:
            quat = 4
        elif np.sign(x) == -1 and np.sign(y) == 1:
            quat = 2
        else:
            quat = 3
            
        if quat not in bounds_quat[camera_index]:
            continue
            
        angle = math.atan(y/x)/math.pi*180
        
        if quat in [1,2] and angle < 0:
            angle += 180
        elif quat in [3,4] and angle > 0:
            angle -= 180
        
        if camera_index == 4:
            if abs(angle) > 145:
                return True
        else:
            if angle >= min(bounds[camera_index]) and angle <= max(bounds[camera_index]):
                return True
            
    
    return False

In [4]:
def check_cam_row(row, camera_index):
    points = [(row.iloc[4+i], row.iloc[8+i]) for i in range(4)]
    for x, y in points:
        
        if np.sign(x) == 1 and np.sign(y) == 1:
            quat = 1
        elif np.sign(x) == 1 and np.sign(y) == -1:
            quat = 4
        elif np.sign(x) == -1 and np.sign(y) == 1:
            quat = 2
        else:
            quat = 3
            
        if quat not in bounds_quat[camera_index]:
            continue
            
        angle = math.atan(y/x)/math.pi*180
        
        if quat in [1,2] and angle < 0:
            angle += 180
        elif quat in [3,4] and angle > 0:
            angle -= 180
        
        if camera_index == 4:
            if abs(angle) > 145:
                return True
        else:
            if angle >= min(bounds[camera_index]) and angle <= max(bounds[camera_index]):
                return True
            
    
    return False

### distance

In [5]:
def get_distance(row):
    x = np.mean(row.iloc[4:8])
    y = np.mean(row.iloc[8:12])
    return np.sqrt(x**2 + y**2)

### weight/height/centerx/centery style annotation

In [6]:
def get_whxy(row):
    x = np.mean(row.iloc[4:8])
    y = np.mean(row.iloc[8:12])
    w = abs(max(row.iloc[4:8]) - min(row.iloc[4:8]))
    h = abs(max(row.iloc[8:12]) - min(row.iloc[8:12]))
    return np.array([x, y, w, h])

### Upper Left corner

In [7]:
def get_scaled(row):
    upper_left_x = (row.loc['center_x'] + 40 - row.loc['box_width']/2) * 612 / 80
    upper_left_y = (80 - (row.loc['center_y'] + 40 + row.loc['box_height']/2)) * 768 / 80
    return np.array([upper_left_x, upper_left_y, row.loc['box_width']*612 / 80, row.loc['box_height']*768 / 80])

### Generate features and save file

In [8]:
def generate_features(anno_path,output_path):
    anno = pd.read_csv(anno_path)
    for index, name in enumerate(camera_name):
        anno[name] = anno.apply(lambda x: check_cam_row(x, index), axis = 1)
    anno['distance'] = anno.apply(lambda x: get_distance(x), axis = 1)
    anno['center_x'], anno['center_y'], anno['box_width'], anno['box_height'] = np.array(anno.apply(lambda x: get_whxy(x), axis = 1).tolist()).T
    anno['scaled_x'], anno['scaled_y'], anno['scaled_box_width'], anno['scaled_box_height'] = np.array(anno.apply(lambda x: get_scaled(x), axis = 1).tolist()).T
    anno.to_csv(output_path)

In [9]:
generate_features('../datasets/dl2020/annotation.csv','../datasets/dl2020/annotation_newfeat_2.csv')

In [10]:
anno_new = pd.read_csv('../datasets/dl2020/annotation_newfeat_2.csv')

In [11]:
anno_new.head()

,Unnamed: 0,scene,sample,action,category,fl_x,fr_x,bl_x,br_x,fl_y,...,back_right,distance,center_x,center_y,box_width,box_height,scaled_x,scaled_y,scaled_box_width,scaled_box_height
0,0,111,0,object_action_driving_straight_forward,car,-14.270185,-14.286638,-9.696476,-9.712929,19.254325,...,False,23.503215,-11.991557,20.213948,4.590162,1.919246,196.707221,180.733722,35.114737,18.424762
1,1,111,0,object_action_parked,car,-5.902891,-5.657694,-1.628863,-1.383666,29.164061,...,False,30.036207,-3.643278,29.814430,4.519225,2.424500,260.842884,86.143868,34.572071,23.275205
2,2,111,0,object_action_walking,pedestrian,37.068518,36.283792,37.300167,36.515442,-20.233128,...,False,41.724720,36.791980,-19.680002,1.016375,1.106252,583.571009,567.618014,7.775271,10.620019
3,3,111,0,object_action_driving_straight_forward,car,1.972478,1.988930,-2.744222,-2.727770,8.132214,...,False,7.181891,-0.377646,7.171955,4.733153,1.920518,285.006699,305.930750,36.208618,18.436969
4,4,111,0,object_action_driving_straight_forward,car,39.051316,39.067769,34.334616,34.351068,5.287296,...,False,36.955389,36.701192,4.327037,4.733153,1.920518,568.659812,333.241961,36.208618,18.436969
